#  1.  Recurrent Neural Networks - introduction

Recurrent Neural Networks (RNNs) are the natural extention of feedforward networks to understanding the input / output relationship between ordered sequences.  In this short notebook we first motivate the use of RNNs by discussing some limitations of supervised learning in terms of the kinds of problems it can tackle.  We then describe the vanilla RNN framework more formally, as well as solution methods and implementation issues.

# 1.  Limitations of supervised learning

With supervised learning - regression and classification - we aim to learn the pattern between a set of fixed size input and ouptput data points.  For example, with object detection (a classification problem) we can use a classification algorithm to learn a model that distinguishes between image patches containing an object of interest (e.g., human face) and all those *not* containing this object.  This classification problem is shown figuratively in the image below (taken from [[1]](#bib_cell)).

<img src="images/wright_bros_face_detect.png" width=600 height=600/>

## Problem 1 - vector input / output

Note one key aspect of this sort of problem we can solve with supervised learning: *the input and output of a supervised learning system are fixed length scalars / vectors.*  With object detection the inputs - image patches - are all the same size e.g., a 64 x 64 grid of pixels and the outputs (vectorized as a $64^2$ x 1 vector)  - labels - are integer valued scalars.  An example of a small image patch showing the value of each pixel (taken from [[2]](#bib_cell)) is shown below.

<img src="images/sample_grid_a_square.png" width=400 height=400/>

But not all pattern recognition problems satisfy this condition on their input / output.  

For example

- An automatic speech recognition program takes in a sequence (a segment of raw audio) and outputs a sequence of letters (e.g., a word or sentence).  


- An automatic lagnuage translator takes in a sequence of words in one language (e.g., English) and outputs a sequence of words in another language (e.g., Spanish). 

 ## Problem 2 - complicated sequential relationships 

A more subtle issue with pattern recognition tasks like e.g., machine translation is that the input / output data sequences have a more complicated relationship than that of a typical supervised learning problem.  Take an English to Spanish translation of the sentence

I do not like cats. --> Los gatos me cae mal.

If we look at this datapoint on a word-by-word level, then it is **not** the case that each word in the two sentences translates directly.  e.g., "I" does not translate correctly to "Los", "do" is not correctly translated as "gatos", etc.,  Moreover "cats" is near the back of the English sentence and near the front ("los gatos") of the Spanish translation.  So while on the whole these two sentences mean the same, it is not the case that each word can be correctly translated in sequence. 

## Summary

>In summary, supervised learning cannot directly tackle pattern recognition problems whose input and output consist of ordered sequences of data.  Popular examples of such problems include speech recognition and machine translation.

# 2.  RNN basic modeling

RNNs are a direct generalization of feedforward networks to ordered sequential data.  This essentially allows us to apply the nonlinear power of feedforward neural nets to this sort of data.

In this Section we introduce notation and formal modeling of the basic RNN model. 

## 2.1  Sequence notation

First we need some notation to denote sequences of input / output.  We can denote one input sequence of data as 

$\mathbf{x}^{\left(1\right)},\,\mathbf{x}^{\left(2\right)},...,\mathbf{x}^{\left(S\right)}$

Here each vector $\mathbf{x}^{\left(t\right)}$
 is of length $L$.  Likewise each corresponding output sequence is denoted as 

$\mathbf{y}^{\left(1\right)},\,\mathbf{y}^{\left(2\right)},...,\mathbf{y}^{\left(S\right)}$

and each output vector $\mathbf{y}^{\left(t\right)}$ has length $K$.


Notice here that while the input and output vectors themselves can have different lengths - $L$ and $K$ respectively - both input and output sequences are of length $S$.  The basic RNN can be adjusted to deal with input and output sequences of different lengths, which is something we discuss later on.

------ 
### Example -  one-hot-encoding vectors for machine translation

In machine translation - and other text-based pattern recognition problems - each word is often represented in a 'one-hot encoding' format.  Take our example English-to-Spanish translation sentence

I do not like cats. --> Los gatos me cae mal.

Each word in the first sentnece is transformed into a long vector with length equal to the number total words in the English dictionary (so this vector can have tens of thousands of entries) ordered alphabetically.  Take the second word in the English sentence - "do".  The one-hot-encoded vector version of this word is all zeros except where the word "do" appears in the dictionary alphabetically (where a 1 is placed).  So e.g., if "do" were the 5,000th word in the dictionary the vector version would be all zeros with a 1 in the 5,000th entry.  This is illustrated figuratively below

$\mathbf{x}^{\left(2\right)}=\left[\begin{array}{c}
0\\
\vdots\\
0\\
1\\
0\\
\vdots\\
0
\end{array}\right]\begin{array}{c}
\\
\\
\\
\longleftarrow\text{index of where "do" is in the English dictionary}\\
\\
\\
\\
\end{array}$


Doing this for each word in the english sentence we get a sequence of 5 input vectors of equal length.

We perform a similar one-hot-encoding of the output: the Spanish words.  However because the Spanish dictionary may be of different length so too can these output vectors be of different length than the input.  In either case, after performing the one-hot-encoding transformation we have a sequence of 5 output vectors as well.

------ 
### Example -  speech recognition

Below is a picture of the waveform for the word "dog" spoken by a native English speaker. 

<img src="images/dog_speech.png" width=600 height=600/>

Such a waveform is typically windowed and transformed into a sequence of spectral slices - called a spectrogram - as illustrated figuratively in the image below (taken from [[1]](#bib_cell)).

<img src="images/spectrogram_creation.png" width=600 height=600/>

Each spectral slice is an input vector to the speech recognition system - and all have the same dimension.  Each spectral slice has a corresponding output - a subcomponent sound of the word "dog" - called a phoneme.  

In this manner speech recognition has an input / output sequence correspondence between the spoken waveform and the word dog. This sort of input / output correspondence holds more generally (for other words, sentences, languages) as well.

##  2.2  From feedforward network to RNN 

RNNs are a natural extension of standard feedforward neural networks - where both input and output are ordered sequences.  

Take a standard single (hidden) layer network with $M$ hidden units that takes as input vectors $\bf{x}$.  The formula for each hidden unit is a nonlinear function like tanh, a logistic sigmoid, or a relu function, e.g., for a single tanh hidden unit the output with respect to one input datapoint can be written as $h_m$ where

$h_{m}=\text{tanh}^{\,}\left(\mathbf{x}^{T}\mathbf{v}_{m}^{\,}\right)$

Abusing notation slightly, we can then write the entire output of this networks hidden layer in vector form as $\bf{h}$

$\mathbf{h}=\text{tanh}\left(\mathbf{V}^{T}\mathbf{x}^{\,}\right)$


Here $\mathbf{V}$ is the matrix consisting of all weight vectors $\mathbf{v}_1,...,\mathbf{v}_M$ stacked together column-wise.

Now the predicted output $\hat{y}$ can be written as

$\hat{y} = \mathbf{u}^T\mathbf{h}^{~} = \sum_{m=1}^{M} u_{m}~\text{tanh}^{\,}\left(\mathbf{x}^{T}\mathbf{v}_{m}^{\,}\right)$

How do we tune the parameters here to make sure our predicted output matches the real output?  By shoving the above through a cost function!  Which cost function we choose depends on the application - e.g., if this is a regression or classification problem.

This network is represented graphically in the left panel of the image below.  In the right panel is a graphical representation of an RNN associated with this feedforward network.

<img src="images/dnn2rnn.png" width=500 height=500/>

Notice the only difference is that *the hidden layer loops in on itself several times recursively*.  How many times does it loop in on itself?  The length of the input / output sequence data, which we have denoted as $S$.  

Translating the picture formally, at the $s^th$ loop the hidden layer output $\mathbf{h}_s$ is defined recursively in terms of $\mathbf{h}_{s-1}$ as 

$\mathbf{h}_s=\text{tanh}\left(\mathbf{V}^{T}\mathbf{x}_s^{\,} + \mathbf{W}^T\mathbf{h}_{s-1}\right)$

and the associated output is then likewise given as 

$\hat{\mathbf{y}}_{s} = \mathbf{U}^T\mathbf{h}_{s}^{~}$

And these are defined for each $s = 1,...,S$. 

Note the similarities between the equations that define a single hidden layer feedforward network, and those that define the analagous RNN.  We have literally just adjusted the feedforward architecture so reasonably so that we can deal with sequential input / output data!

How do we tune the parameters here to make sure our predicted output sequence matches the real output sequence?  By shoving the above through a cost function!  Which cost function we choose depends on the application - e.g., if this is a regression or classification problem.

## 2.3.  Parameter tuning using gradient descent

As with feedforward networks we use gradient descent (a.k.a. backpropogation) to tune the parameters of an RNN.  However one big technical hurdle that arises when applying gradient descent to feedforward nets (using e.g., logistic sigmoid activation functions) - the *gradient vanishing problem* - arises as well with RNNs.  

Using a different activation function - i.e., relu and its offspring - greatly allievates this issue with feedforward networks, and offer similar relief with RNNs.  However a more commonly applied remedy with RNNs is to slightly adjust the architecture of recursive RNN loop itself.  There are an array of alterations one can make here to the architecture, but by far the most popular is the [Long Term Short Memory (LTSM)](https://en.wikipedia.org/wiki/Long_short-term_memory) RNN.  

For a nice visual introduction to LTSMs check out [4].

<a id='bib_cell'></a>

## Bibliography

[1] Watt, Jeremy et al. [Machine Learning Refined](www.mlrefined.com). Cambridge University Press, 2016

[2] Image taken from http://pippin.gimp.org/image_processing/chap_dir.html

[3] Graves, Alex. "Supervised sequence labelling." Supervised Sequence Labelling with Recurrent Neural Networks. Springer Berlin Heidelberg, 2012. 5-13.

[4] http://colah.github.io/posts/2015-08-Understanding-LSTMs/